In [1]:
import pandas as pd
import os
import requests
from pprint import pprint
from config import g_key
import gmaps
import numpy as np

In [2]:
path = os.path.join('..','WeatherPy','cities_df.csv')
df=pd.read_csv(path)
df.head()

,Cities,Country,Latitude,Longitude,Cloudiness,Date,Humidity,Max_Temp,Wind Speed
0,takoradi,GH,4.88,-1.76,89,1600045628,92,75.69,7.23
1,ulladulla,AU,-35.35,150.47,40,1600045629,55,66.20,8.05
2,caravelas,BR,-17.71,-39.25,0,1600045629,84,72.37,8.95
3,kaitangata,NZ,-46.28,169.85,64,1600045261,59,62.01,5.99
4,cairo,EG,30.06,31.25,6,1600045519,78,78.80,9.17


In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations=df[['Latitude','Longitude']]
humidity=df['Humidity'].astype(float)
maxx=df['Humidity'].max()
#maxx=maxx.astype(float)

In [5]:
fig=gmaps.figure()

In [6]:
heat_layer=gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=maxx,point_radius=0.9)
fig.add_layer(heat_layer)

In [7]:
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
#Finding ideal conditions
df_hotels=df[(df['Max_Temp']>70) & (df['Max_Temp']<80) & (df.Cloudiness == 0) & (df['Wind Speed']<10)]

In [9]:
df_hotels.shape

(18, 9)

In [10]:
df_hotels.reset_index(inplace=True,drop=True)

In [11]:
df_hotels

,Cities,Country,Latitude,Longitude,Cloudiness,Date,Humidity,Max_Temp,Wind Speed
0,caravelas,BR,-17.71,-39.25,0,1600045629,84,72.37,8.95
1,santa cruz,BO,-17.80,-63.17,0,1600045594,74,78.80,5.82
2,santa helena de goias,BR,-17.81,-50.60,0,1600045633,26,72.84,2.73
3,saint-louis,SN,16.33,-15.00,0,1600045658,83,78.80,2.24
4,port said,EG,31.26,32.28,0,1600045663,83,78.80,6.93
5,abnub,EG,27.27,31.15,0,1600045665,44,78.80,9.17
6,byron bay,AU,-28.65,153.62,0,1600045635,60,73.99,3.36
7,awbari,LY,26.59,12.78,0,1600045674,31,76.60,4.79
8,victoria point,AU,-27.58,153.30,0,1600045688,70,79.00,3.00
9,toowoomba,AU,-27.55,151.97,0,1600045695,56,72.00,3.36


In [12]:
# target_location = "31.34,69.45"
# target_radius=5000
# target_keyword="Hotel"
# target_type="Hotel"

In [13]:
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params={"location":target_location,"radius":target_radius,"type":target_type,"keyword":target_keyword,"key":g_key}

In [14]:
#response = requests.get(base_url,params=params).json()

In [15]:
params={"radius":5000,"type":"hotel","keyword":"hotels","key":g_key}
for index,row in df_hotels.iterrows():
    print("Searching for {}".format(row['Cities']))
    lat=row['Latitude']
    lon=row['Longitude']
    params['location'] = f'{lat},{lon}'
    print(params['location'])
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    
    response=requests.get(base_url,params=params).json()
    
    try:
        df_hotels.loc[index,'Hotel_Name']=response['results'][0]['name']
    except KeyError:
        print("Couldn't find results")
    except IndexError:
        print("No hotels were found in the vicinity")

Searching for caravelas
-17.71,-39.25


C:\Users\Pankaj\Anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Pankaj\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Searching for santa cruz
-17.8,-63.17
Searching for santa helena de goias
-17.81,-50.6
Searching for saint-louis
16.33,-15.0
No hotels were found in the vicinity
Searching for port said
31.26,32.28
Searching for abnub
27.27,31.15
No hotels were found in the vicinity
Searching for byron bay
-28.65,153.62
Searching for awbari
26.59,12.78
No hotels were found in the vicinity
Searching for victoria point
-27.58,153.3
Searching for toowoomba
-27.55,151.97
Searching for touros
-5.2,-35.46
Searching for asyut
27.18,31.18
Searching for laguna de duero
41.58,-4.72
Searching for port macquarie
-31.43,152.92
Searching for sawtell
-30.38,153.1
Searching for tigzirt
36.89,4.12
Searching for gat
31.61,34.76
Searching for zhuanghe
39.7,122.99


In [16]:
df_hotels

,Cities,Country,Latitude,Longitude,Cloudiness,Date,Humidity,Max_Temp,Wind Speed,Hotel_Name
0,caravelas,BR,-17.71,-39.25,0,1600045629,84,72.37,8.95,Mar Doce de Abrolhos
1,santa cruz,BO,-17.80,-63.17,0,1600045594,74,78.80,5.82,Marriott Santa Cruz de la Sierra Hotel
2,santa helena de goias,BR,-17.81,-50.60,0,1600045633,26,72.84,2.73,Castro Castle Hotel
3,saint-louis,SN,16.33,-15.00,0,1600045658,83,78.80,2.24,NaN
4,port said,EG,31.26,32.28,0,1600045663,83,78.80,6.93,فندق رستا بورسعيد Resta Port Said Hotel
5,abnub,EG,27.27,31.15,0,1600045665,44,78.80,9.17,NaN
6,byron bay,AU,-28.65,153.62,0,1600045635,60,73.99,3.36,Elements Of Byron Resort & Spa
7,awbari,LY,26.59,12.78,0,1600045674,31,76.60,4.79,NaN
8,victoria point,AU,-27.58,153.30,0,1600045688,70,79.00,3.00,Seminara Apartments
9,toowoomba,AU,-27.55,151.97,0,1600045695,56,72.00,3.36,Quest Toowoomba


In [17]:
df_hotels.dropna(inplace=True)

C:\Users\Pankaj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
df_hotels.reset_index(inplace=True,drop=True)

In [19]:
df_hotels

,Cities,Country,Latitude,Longitude,Cloudiness,Date,Humidity,Max_Temp,Wind Speed,Hotel_Name
0,caravelas,BR,-17.71,-39.25,0,1600045629,84,72.37,8.95,Mar Doce de Abrolhos
1,santa cruz,BO,-17.80,-63.17,0,1600045594,74,78.80,5.82,Marriott Santa Cruz de la Sierra Hotel
2,santa helena de goias,BR,-17.81,-50.60,0,1600045633,26,72.84,2.73,Castro Castle Hotel
3,port said,EG,31.26,32.28,0,1600045663,83,78.80,6.93,فندق رستا بورسعيد Resta Port Said Hotel
4,byron bay,AU,-28.65,153.62,0,1600045635,60,73.99,3.36,Elements Of Byron Resort & Spa
5,victoria point,AU,-27.58,153.30,0,1600045688,70,79.00,3.00,Seminara Apartments
6,toowoomba,AU,-27.55,151.97,0,1600045695,56,72.00,3.36,Quest Toowoomba
7,touros,BR,-5.20,-35.46,0,1600045696,83,75.20,8.05,Hotel Vila Gale Touros
8,asyut,EG,27.18,31.18,0,1600045698,44,78.80,9.17,Assiut hotels Armed Forces
9,laguna de duero,ES,41.58,-4.72,0,1600045700,37,71.60,6.93,AC Hotel by Marriott Palacio de Santa Ana


In [20]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df_hotels.iterrows()]
locations = df_hotels[["Latitude", "Longitude"]]

In [21]:
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)

In [22]:
fig.add_layer(marker_layer)


In [23]:
fig

Figure(layout=FigureLayout(height='420px'))